# Clustering of websites by text

Thanks to the scrapping notebook I could gather the text of the given websites and the links in the homepage and write it into a text file. 

There is one text file per site. 

While most of the 50 sites were scrapped succesfully, there a few I couldn't get. 

Out of 50 sites:

2 couldn't be requested properly.
5 gave a 403 or 503 http error code. 
1 was requested, but the script recovered the text of the loading page. 

I will start the clustering with the data I could gather, and go back to the missing websites after the clustering process is well stablished.

My first step is to load the text data and load it into a dataframe, in order to start exploring it. 

In [ ]:
import os
import re

import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns


%matplotlib inline

In [ ]:
# Set the contents directory
CONTENTS_DIR = './site_contents/'

# Get all file names from the directory.

file_names = [file for file in os.listdir(CONTENTS_DIR)]

# Read the text of each file
file_contents = []

for name in file_names:
    
    with open(CONTENTS_DIR + name, 'r') as content:
        site_text = content.read()
    
    file_contents.append(site_text)

In [ ]:
# Build a dataframe with the site names and the extracted text.
websites_df = pd.DataFrame({'site': map(lambda name: name.replace('.txt','' ), file_names),
                            'raw_text': file_contents})

# Some info on the dataframe.

websites_df.info()

We can have a look a the Dataframe to see what it looks like.

In [ ]:
websites_df.head(5)

We see that there is an entry with empty text. 
We also expect to find some entries with very few words, which may not be enough to perform a classification reliably. 

A way to filter out these cases is to remove all entries whose text is shorter than a defined cutoff. 

We add a wordcount column to the dataframe in order to get this information.

Note that this wordcount is only a rough first approximation, as text has not been treated yet, and some words will be removed afterwards. Still, it is useful for filtering some of the entries. 

In [ ]:
# Add column with split text, and one with the length of the split text.
websites_df['wordcount']=websites_df['raw_text'].apply(lambda mytext: len(mytext.split()))


In [ ]:
# Statistics on the wordcount.
print(websites_df['wordcount'].describe())
# Lets see how the wordcount is distributed and plot an histogram.
# I add +1 to the wordcount in order to avoid computing log(0).


websites_df['wordcount'].apply(lambda wc: np.log10(wc +1)).hist(bins = 20, figsize=(7,7))
plt.title('Word count distribution')
plt.xlabel('Log10(wordcount)')
plt.show()

Apart from 2 sites, most of our texts have more than 100 words. We can remove the oones that don't reach this value.

In [ ]:
# Have a look at the shortest entries.
websites_df.sort_values('wordcount').head(10)

In [ ]:
# Set minimum amount of words to remain in the dataset.

min_words = 100

# Drop short texts
websites_df.drop(websites_df[websites_df['wordcount']< 100].index, inplace=True)

websites_df.info()

In [ ]:
# Lets see how the wordcount is distributed now.

websites_df['wordcount'].apply(np.log10).hist(bins = 20, figsize=(7,7))
plt.title('Word count distribution')
plt.xlabel('Log10(wordcount)')
plt.show()

In [ ]:
print(f'After removal, we have {websites_df.shape[0]} sites left.')

We still have very different lenghts, but even a few hundred words could help us classify a site. 

## Vectorizing text with Tf-Idf

I will use Tf-Idf to vectorize my texts. This will build a vocabulary, and assing a vector to each document. The i-th coordinate of the vector is the product between the Term Frequency and the Inverse Document Frecuency of the i-th word in the vocabulary.

Furthermore, this vectorizer will normalize all vectors to length 1, which will reduce the efect of comparing texts of very different lengths. 

In [ ]:
# Get stopwords for all languages included in nltk
print(f'Compile a list of stopwords from the {len(stopwords.fileids())} languages in nltk: \n')
print(', '.join(stopwords.fileids()))

# Build list of all stopwords:

stop_words = []
for lan in stopwords.fileids():
    stop_words.extend(stopwords.words(lan))

# Get a list of curated html tags that may have remained.
# The list contains all html tags, and the ones that shouldn't be included
# as stopwords are preceded by the simbol '#'.

print('\nInclude curated HTML tags and common file extensions.\n')
with open('filter_tags', 'r') as tags:
    stop_words.extend([tag.strip() for tag in tags.readlines() if (not tag.startswith('#') and len(tag.strip())>=3)])

# Let's add to the stopwords a list of file types and html tags, in case something got through

file_types = ['pdf', 'jpg', 'jpeg', 'png', 'gif', 'exe', 'js', 'zip', 'tar', 'gz', '7z', 'rar']

stop_words.extend(file_types)

# Remove repetitions

stop_words = list(set(stop_words))
print(f'Gathered a total of {len(stop_words)} unique stopwords.')

In [ ]:
def remove_numbers_lower(text):
    """
    Preprocessing function to be passed to the tokenizer. 
    It will remove numbers that appear by themselves of between '-'.
    This removes things such as telephone numbers. 
    It also lowecases the entirer string.
    
    text: string. Full text to be treated. 
    """
    
    processed = re.sub(r'\b\d+\b', '',text)
    processed = processed.lower()
    
    return processed

In [ ]:
# Creating a vectorizer object. We can pass parameter to control certain aspects
# of the vectorization process.

min_freq = 2 # parameter to remove very uncommon words.
max_freq = 0.4 # parameter to remove too common words.

# Changed token patterns to keep words with  or more letters only.
min_letters = 3
my_tokens = '(?u)\\b\\w{'+ str(min_letters -1) + '}\\w+\\b'

vectorizer = TfidfVectorizer(min_df = min_freq,
                             max_df = max_freq,
                             preprocessor= remove_numbers_lower,
                             stop_words=stop_words,
                             token_pattern = my_tokens,
                             ngram_range=(1,1)
                            )

In [ ]:
# Building vocabulary and generating the document term matrix (dtm).

dtm = vectorizer.fit_transform(websites_df['raw_text'])

print(f'We obtained a vocabulary of {len(vectorizer.vocabulary_)} different words.')

In [ ]:
print(f'We identified {len(vectorizer.stop_words_)} words to be ignored based on'
      + f' max_df= {max_freq} and min_df = {min_freq}' )

#print('\n\n' + ' | '.join(vectorizer.stop_words_))

In [ ]:
# Build an inverse vocabulary dictionary to retrieve words easily. 

inv_vocab = {  w_id: word  for word, w_id in vectorizer.vocabulary_.items() }

The vectors obtained in this way are stored in a sparse matrix (i.e. it's entries are mostly 0).

Each row corresponds to a document, and each column to a word in the vocabulary.

In [ ]:
print(f'The dimensions of the document matrix are {dtm.shape}'
      + f', and it has {dtm.nnz} non-zero elements .')
print(f'That represents {dtm.nnz * 100/np.multiply(*dtm.shape):.2f} percent of the entries')

###  Visualizing the document vectors

With the documents matrix built, we can explore it superficially in order to get an idea of the structures of our documents. 

We show two visualizations: The non-zero elements of the matrix, and the cosine similarity matrix. 

In [ ]:
# Visualizing the non-zero elements of the matrix. 

plt.figure(figsize=(10, 5))
plt.spy(dtm, markersize=1, aspect = 'auto')
plt.title('Document matrix non-zero elements')
plt.xlabel('Word in vocabulary')
plt.ylabel('Document number')
plt.show()

In [ ]:
# Computing cosine similarities. Since verctors are normalized to 1, 
# it suffices to multiply the matrix by its transpose. 

cosine_sims = dtm * dtm.transpose()

# Heatmap of the cosine similarities. 
# There are no correletions higher than 0.6 (other than the diagonal), 
# So we choose this value for a cutoff of the scale.

my_cmap = cm.get_cmap('viridis')
plt.figure(figsize=(8,8))
plt.imshow(cosine_sims.toarray(), cmap=my_cmap, vmax=.8)
plt.colorbar(shrink = 0.8)

plt.title('Cosine Similarities between documents')

plt.show()

Although clusters are hard to identify in this plot, we see that ther might be some relation between sites number 3,4 and 5. Let's check what they are. 

In [ ]:
websites_df.iloc[2:5]

The sites that seem correlated are:
* de.jointeflons.com
* de.plasticptfe.com
* german.aiflon.com

We can see that they are all german sites, and at least the first two are related to PTFE (teflon) products.

This ilustrates that cosine similarity can capture some of the similarities between sites. 

##  Dimensional reduction

Reducing dimensionality can help the KMeans algorithm work better, as it relies on euclidean distances. 

The price to pay for this is that we may loose some interpretability, as the components of our dimensions will be mixed. 

The question that arises is deciding how many dimensions to keep. A common criterium is is to plot the singular values obtained (scree plot), and keep the dimensions corresponding to find the elbow in the plot. 

We will keep the first 20 principal components and see if the elbow becomes aparent. It should be enough.

NOTE: I consider this number of dimensions as in hyperparameter, similar to the max and min frequencies passed to the vectorizer. Of course, with a list of 50 sites, this choice wont necesarily be representative for a general case. But with a more representative sample of sites the same procedure could be followed. 

Once we determine the number of dimensions to keep, we will then perform KMeans on the dimensionally reduced vectors. 

In [ ]:
# We need to use truncated SVD because we are dealing with an sparse matrix.

# Create the SVD object. 

decomposer_20d = TruncatedSVD(20, n_iter=10)

In [ ]:
# Perform SVD/LSA and get the transformed doc vectors. 
docs_20d = decomposer_20d.fit_transform(dtm)

Now we plot the singular values, and look for the elbow in the plot.

In [ ]:
plt.figure(figsize=(8,8))

singular_values = decomposer_20d.singular_values_
plt.plot(singular_values)
plt.grid(True)
plt.xticks(list(range(len(singular_values))))
plt.title('Scree plot - Singular Values')
plt.xlabel('Dimension index')
plt.ylabel('Singular Value')
plt.show()

It appears that in this plot the elbow correspond to the 7th singular value (indices start from 0).

But let's use the helper method to detect the elbow.

In [ ]:
dimension_number = np.array(list(range(len(singular_values))))

elbow_dim_number = ClusterDecissionHelper.elbow_finder(dimension_number, singular_values, plot=True )

print(f'Found elbow for dim. with index {elbow_dim_number}.')
print(f'We should truncate to {elbow_dim_number+1} dimensions! (remember, indices start from 0)')

### Interpretation

We can see what are the most relevant words for these 7 dimensions.

In [ ]:
pc_dict = {}

for i in range(7):
    pc = decomposer_20d.components_[i, :] #get principal component
    
    # Get arguments sorted in decreasing order. 
    argsorted = np.argsort(pc)[::-1]
    
    # Find words in vocabulary. 
    pc_dict[f'pc_{i}'] =  [inv_vocab[i] for i in argsorted[:10] ]

In [ ]:
# Print words
print("Principal components words orderd by importance. \n")
for k,v in pc_dict.items():
    print(f'Principal component {k} : \n')
    print(' | '.join(v) + '\n')

There seems to be some overlapping between the componenets. 

### Visualization

Principal componenets can be used to visualize our data, plotting each point according to it's first two components. 

In [ ]:
def plot_pc(vectors, components = [0,1], title = None, **kwargs):
    """
    Scatter plot of the principal components specified by the columns.
    vectors: array-like or sparse matrix of size (n_samples * n_components).
    components: list or tuple with the two directions to take as x,y.
    title: string. Plot title.
    kwargs for plt.scatter.
    """
    plt.figure(figsize=(8,8))
    
    col_x, col_y = components

    x = vectors[:,col_x]
    y = vectors[:,col_y]
    plt.scatter(x,y, **kwargs)
    
    if title:
        plt.title(title)
    
    plt.xlabel(f'Component {col_x}')
    plt.ylabel(f'Component {col_y}')

    plt.show()
    

In [ ]:
plot_pc(docs_20d[:,:2], title='Sites')

Points are more or less divided along the two directions, but there is no obvious clustering in sight.

## Selecting K from silhouette score

We perfom clustering with different number of clusters and compare silhouette scores. The number of clusters that gives the highest silhouette score is considered the best.
The process is repeated several times, and we retain the number of clusters that provided a max silhouette more times.

In [ ]:
class ClusterDecissionHelper():
    """
    Suggest and optimal number of clusters K for KMeans based on their scores.
    
    This class allows to perform KMeans with different numbers of clusters and compute 
    the silhouette score, inertia, or both. Repeats the process several times, and analize
    the scores for the optimal K. 
    """
    
    def __init__(self, max_clusters, min_clusters = 2,
                 repeat = 5, metric = 'euclidean', **kwargs ):
        
        """
        kwargs: pass arguments to sklearn.cluster.KMeans()
        min_clusters: int. Lower bound for the number of clusters. Default is 2, the minimum
                   allowed value. 
        max_clusters: int. Upper bound for the number of clusters.
                      Maximum allowed value is (number of vectors - 1)
        """
        
        self.repeat = repeat
        self.metric = metric
        
        self.KMeans_kwargs = kwargs
        
        self.cluster_numbers = list(range(min_clusters,max_clusters+1))
        
        self.sil_scores = None
        self.inertia_scores = None
        
        
    def compute_scores(self, dt_matrix,
                       sil_score = False, inertia = False):
    
        """
        Computes arrays of the desired score, of size self.repeat x len(self.cluster_numbers)
        It need to be told which method to use.


        dt_matrix: array-like or sparse matrix. Document term matrix with docs to be clustered.
        sil_score: bool. Compute Silhouette score. Default is False. 
        inertia: bool. Compute inertia. Default is False.    
        """

        repeat = self.repeat
        cluster_numbers = self.cluster_numbers
        #metric = self.metric

        doc_number = dt_matrix.shape[0]

        # Build array to keep the results of each run
        sil_array = np.zeros((repeat, len(cluster_numbers)))
        inertia_array = np.zeros((repeat, len(cluster_numbers)))

        for c_idx,c_number in enumerate(cluster_numbers):

            model = KMeans(n_clusters=c_number, **self.KMeans_kwargs)

            for repetition in range(repeat):
                # Fit the model and get the labels
                model.fit(dt_matrix)
                labels = model.labels_

                # Compute scores

                if sil_score:
                    sil_array[repetition, c_idx] = silhouette_score(dt_matrix, labels, metric = self.metric)
                if inertia:
                    inertia_array[repetition, c_idx] = model.inertia_

        if sil_score:
            self.sil_scores = sil_array 
        if inertia:
            self.inertia_scores = inertia_array
            
    def get_K_from_silhouettes(self):
        """ 
        Identifies optimal K from silhouette scores obtained with compute_scores()
        If two or more values of K are tied, the smallest one is selected.
        """
        
        scores = self.sil_scores

        # The -1 factor ensures that if two or more values tie, the smaller one
        # is retained. 
        ranked_K_indices = np.argsort(-1*np.bincount(scores.argmax(axis=1)), kind='stable')

        winning_K = self.cluster_numbers[ranked_K_indices[0]]
        
        self.best_K_sil = winning_K

        return winning_K
    
    def plot_scores(self, method = 'sil'):
        """Convinience method for plotting scores obtained with compute_scores().
        method: string. 'sil' for silhouettes, or 'in' for inertia. 
        """

        titles = {'sil': 'Silhouette score', 'in': 'Inertia' }

        scores = {'sil': self.sil_scores, 'in':self.inertia_scores}
        
        score_arr = scores[method]
        title = titles[method]

        fig, ax = plt.subplots(figsize = (8,8))

        for i in range(self.repeat):
            ax.plot(self.cluster_numbers, score_arr[i,:])
            
        plt.xticks(self.cluster_numbers)
        plt.xlabel('Cluster Number')
        plt.ylabel('Score')
        plt.title(f'{title} for {self.repeat} repetitions.')

        plt.grid(True)

        plt.show()
        
    @staticmethod
    def elbow_finder(x,y, plot = False):
        """Finds an elbow in the graph of y=f(x) by finding the maximum distance between
        f(x) and the line that joins (x_0, y_0) with (x_final, y_final).
        Depends strongly on the noise.
        x,y: One dimensional arrays of values.
        """
        
        # initial point
        pi = np.array([x[0], y[0]])
        
        # final point
        
        pf = np.array([x[-1], y[-1]]) 
        
        # unit vector parallell to the line
        
        n_vec = (pf - pi) / np.linalg.norm((pf-pi))
        
        distances = np.zeros(len(x))
        
        # I don't need the inital and final point. Distance is 0 by definition.
        for i in range(len(x[1:])):
            
            q =np.array([x[i],y[i]])
            
            dif_vec= pi-q
            
            d = np.linalg.norm( (dif_vec) - np.dot(dif_vec,n_vec) * n_vec )
            
            distances[i] = d
            
            elbow_x_id = distances.argsort(kind='stable')[::-1][0]
            
            elbow_x = x[elbow_x_id]
            
        if plot:
            
            plt.figure(figsize=(8,8))
            plt.plot(x, y)
            plt.plot(x[[0,-1]], y[[0,-1]], c = 'g')
            plt.axvline(elbow_x, c='r')
            
        return elbow_x

## KMeans Clustering

We begin by building our 7 dimensional vectors.

In [ ]:
dim_reducer = TruncatedSVD(n_components = 7, n_iter= 10 )

reduced_dtm =  dim_reducer.fit_transform(dtm)

Select K

In [ ]:
decider = ClusterDecissionHelper(max_clusters=20)

In [ ]:
decider.compute_scores(reduced_dtm)

In [ ]:
decider.compute_scores(reduced_dtm, sil_score=False, inertia=True)

In [ ]:
decider.plot_scores('in')

In [ ]:
decider.plot_scores()

In [ ]:
best_K = decider.get_K_from_silhouettes()

print(f'The best value of K found was K = {best_K} .')

In [ ]:
# Create a model for the best K

KMeans_model = KMeans(n_clusters = best_K, n_init=20, max_iter= 100)

In [ ]:
# Find the clusters
KMeans_model.fit(reduced_dtm)

# Write the labels into the dataframe.
websites_df['cluster_label'] =  KMeans_model.labels_

# How many elements in each cluster?
websites_df.groupby('cluster_label')['site'].count()

In [ ]:
# Let's see cosine simlarities ordered by cluster

sorted_vecs = vectorizer.transform(websites_df.sort_values('cluster_label')['raw_text'])


In [ ]:
ordered_indices = websites_df.sort_values('cluster_label')['cluster_label']
n_sites = websites_df.shape[0]
clabel = np.zeros((n_sites, n_sites), dtype = int) -1

for i in range(n_sites):
    for j in range(n_sites):
        if ordered_indices.iloc[i] == ordered_indices.iloc[j]:
            clabel[i,j] = ordered_indices.iloc[i]

In [ ]:
# Sorted cosine sims
sorted_sims = sorted_vecs * sorted_vecs.transpose()

my_cmap = cm.get_cmap('viridis')
plt.figure(figsize=(15,15))
plt.imshow(sorted_sims.toarray(), cmap=my_cmap, vmax=0.6)
plt.colorbar(shrink = 0.8)

for i in range(n_sites):
    for j in range(n_sites):
        if clabel[i, j] >=0:
            text = plt.text(j, i, clabel[i, j],
                           ha="center", va="center", color="w")

plt.title('Cosine Similarities between documents')

plt.show()


In [ ]:
KMeans_model.labels_

In [ ]:
websites_df[websites_df['cluster_label']==6]

In [ ]:
def interpret_clusters(fitted_model, inv_vocabulary, reducer = None, n_words = 10):
    """
    Gets the first n most important words corresponding to each cluster centroid. 
    It needs the fitted KMeans object, and and inverse dictionary built from the vectorizer to look
    for words by id.     
    If dimensional reduction was performed, we also need the TSVD object used for dimensional reduction.
    
    Returns a pandas Dataframe with words and weights for each cluster.
    """
    inv_vocab = inv_vocabulary
    
    if reducer is None:
        centers = fitted_model.cluster_centers_
    else:
        centers = reducer.inverse_transform(fitted_model.cluster_centers_)
    
    #words = {}
    #weights = {}
    cluster_dict = {}
    for i in range(centers.shape[0]):
        vec = inversed_centers[i,:]
        argsorted = np.argsort(vec)[::-1]

        cluster_dict[f'Cluster_{i}'] = {'words': [inv_vocab[i] for i in argsorted[:n_words]], 
                                        'weights': vec[argsorted[:n_words]] }
        
        #words[f'Cluster_{i}'] = [inv_vocab[i] for i in argsorted[:n_words]]
        #weights[f'Cluster_{i}'] = vec[argsorted[:n_words]] 
        
        
    return pd.DataFrame(cluster_dict).transpose()
        

In [ ]:
cluster_df = interpret_clusters(KMeans_model, inv_vocab, dim_reducer)

In [ ]:
cluster_df

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=45)
plt.bar(cluster_df['words'][0], cluster_df['weights'][0])